In [1]:
import pandas as pd
import requests
import json
import time
sleep=0.4

In [2]:
LOCAL=True
if LOCAL:
    f=open('data/inspire_LA.json','r')
    df=json.load(f)
    f.close()
else:
    r=requests.get('https://github.com/restrepo/inspireauth/raw/main/data/inspire_LA.json')
    df=r.json()

In [3]:
print(len(df))

57588


## Fix bad `profile_id` for `LOCAL → True`

In [4]:
#author_id='J.S.Useche.1'
def get_profile_id(author_id,sleep=0.4):
    #if True:
    status=False
    l=[]
    pi=None
    #try:
    #if True:
    try:
        #https://inspirehep.net/literature?q=ea+J.D.Ruiz.Alvarez.2+and+ac+1->10
        url=f'https://inspirehep.net/api/literature?q=ea+{author_id}+and+ac+1->10'
        r=requests.get(url)
        time.sleep(sleep)
        if r.status_code==200:
            data=r.json()
            status=True
            #if not `try` fails and `pass``.
        for d in data.get('hits').get('hits'):
            #l=[a.get('record') for a in d.get('metadata').get('authors') if a.get('bai')==author_id]
            try: #failsafe
                l=[a for a in data.get('hits').get('hits')[0].get('metadata').get('authors') if a.get('bai')==author_id]
            except:
                l=[]
        if len(l)==1 and l[0].get('record'):
            pi=l[0].get('record').get('$ref').split('/')[-1]
    except:
        pass
    return pi,status

TEST=True
if TEST:
    assert get_profile_id('J.S.Useche.1')==('2572877',True)
    assert get_profile_id('J.D.Ruiz.Alvarez.2')==('1184534',True)
    assert get_profile_id('S.Das.8')==(None,True)
    assert get_profile_id('M.Sanchez.39')==(None, True)

In [5]:
def get_advisor(i,sleep=0.4):
    adv=[]
    try:
        r=requests.get('https://inspirehep.net/api/authors/{}'.format(i))
        time.sleep(sleep)
        adv=r.json().get('metadata').get('advisors')
        if adv is None:
            adv=[]
    except:
        pass
    return adv

In [11]:
#a=df[0]
#a.get('author_id')
GET_ADVISORS=False
if GET_ADVISORS:
    kk=[a.update({'advisors':get_advisor(a.get('profile_id'))}) for a in df if not a.get('advisors')]

In [ ]:
SAVE=False
status=0
step=1000
i=0
for start in range(0,len(df),step):
    for d in df[start:start+step]:
        i+=1
        print(str(i).zfill(5),status,end='\r')

        if len(set([dd.get('author_id') for dd in df  if dd.get('profile_id')==d.get('profile_id')]))==1:
            continue
        author_id=d.get('author_id')
        check_profile_id=get_profile_id(author_id)
        if check_profile_id[1] and check_profile_id[0]!=d.get('profile_id'):
            d['profile_id']=check_profile_id[0]
            d['advisors']=get_advisor(d.get('profile_id'))
            status=1
        else:
            status=0
    if SAVE:
        f=open('data/inspire_LA.json','w')
        json.dump(df,f)
        f.close()

## Students object

In [6]:
[dd for dd in df if dd.get('author_id')=='M.Sanchez.39']

[{'author_id': 'M.Sanchez.39',
  'country': 'Colombia',
  'email_addresses': None,
  'full_name': 'Sánchez, M.',
  'ids': None,
  'institution': 'Antioquia U.',
  'institution_id': '903906',
  'name': None,
  'papers': [{'recid': 1415059,
    'year': '2015',
    'citation_count': 2,
    'primary_arxiv_category': [],
    'journal_id': '1365972',
    'author_id': 'M.Sanchez.39',
    'instituion_id': '903906'}],
  'positions': None,
  'profile_id': None,
  'advisors': []}]

In [6]:
df=pd.DataFrame(df)

In [7]:
df[df.advisors.apply(len)>0].shape

(11326, 12)

In [8]:
df[ (df['positions'].str[0].str['rank'].apply(lambda r: r is not None)) & 
   (df['positions'].str[0].str['rank'].apply(lambda r: r is not None)) ].shape

(13745, 12)

In [9]:
df[df['positions'].str[0].apply(lambda r:isinstance(r,dict))].shape

(28388, 12)

People which have edited their profile in INSPIREHEP

In [10]:
df[ (df['positions'].str[0].str['rank'].apply(lambda r:isinstance(r,str))) 
    | (df.advisors.apply(len)>0 ) ].shape

(16762, 12)

In [11]:
df.shape

(57588, 12)

In [12]:
#df['profile_id']=df['profile_id'].fillna(0).astype(int).astype(str).apply(lambda p: None if p=='0' else p)

In [13]:
df[df['profile_id']=='991924']

,author_id,country,email_addresses,full_name,ids,institution,institution_id,name,papers,positions,profile_id,advisors
4722,D.Restrepo.1,Colombia,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...",Antioquia U.,903906,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
4762,D.Restrepo.1,Italy,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...","ICTP, Trieste",902861,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
5942,D.Restrepo.1,Spain,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego A.","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...",Valencia U.,903319,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
6064,D.Restrepo.1,Brazil,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...","IIP, Brazil",912344,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
6066,D.Restrepo.1,Brazil,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, Diego","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...",Campinas State U.,902714,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."
6072,D.Restrepo.1,Spain,"[{'value': 'restrepo@udea.edu.co', 'current': ...","Restrepo, D.A.","[{'value': 'D.Restrepo.1', 'schema': 'INSPIRE ...","Valencia U., IFIC",907907,"{'value': 'Restrepo Quintero, Diego Alejandro'...","[{'recid': 926166, 'year': '2012', 'citation_c...","[{'rank': 'SENIOR', 'hidden': False, 'record':...",991924,"[{'ids': [{'value': 'INSPIRE-00133260', 'schem..."


In [174]:
a={'ids': [{'value': 'INSPIRE-00133260', 'schema': 'INSPIRE ID'}],
 'name': 'Valle, Jose W.F.',
 'hidden': False,
 'record': {'$ref': 'https://inspirehep.net/api/authors/985058'},
 'degree_type': 'phd',
 'curated_relation': False}
r={'author_id':'D.Restrepo.1',
   'name':{'value': 'Restrepo Quintero, Diego Alejandro',
   'preferred_name': 'Diego Restrepo'},
   'positions': [{'rank': 'PHD',
                  'hidden': False,
                  'record': {'$ref': 'https://inspirehep.net/api/institutions/907907'},
                  'current': False,
                  'end_date': '2001',
                  'start_date': '1997',
                  'institution': 'Valencia U., IFIC',
                  'curated_relation': True}
                ]
   }
advisor_profile_id='985058'
d={}

def get_students(r,a):
    ds={}
    ds['degree_type']=a.get('degree_type')
    for p in r.get('positions'):
        if p.get('rank') and p.get('rank').lower()==ds.get('degree_type'):
            for k in p.keys():
                if k=='record':
                    try:
                        ds['institution_id']=p[k].get('$ref').split('/')[-1]
                    except:
                        ds['institution_id']=None
                else:
                    ds[k]=p[k]
    ds['student_id']=r.get('author_id')
    ds['student_name']=r.get('name').get('value')
    ds['student_profile_id']=profile_id
    return ds
assert get_students(r,a).get('institution_id')=='907907'

def get_advisor(d,advisor_profile_id,a,df):
    d['profile_id']=advisor_profile_id
    #d['institution']=r.get('institution')
    #d['institution_id']=r.get('institution_id')

    d['name']=a.get('name')

    try:
        if not d.get('profile_id'):
            d['profile_id']=df[ (df['name'].str['value']==d.get('name')) & 
                        (df['institution_id']==ds.get('institution_id')) 
                        ].iloc[0].get('author_id')
    except:
        pass
    return d
assert get_advisor(d,advisor_profile_id,a,df).get('profile_id')=='985058'

In [175]:
#991924  # J Valle advisor
#1854904
#1045947
#1014628 # J Valle advisor
# 1028906
profile_id=['1014628','1014628','991924','1045947'][2]
r=df[df['profile_id']==profile_id].iloc[0].to_dict()

In [176]:
r.get('advisors')

[{'ids': [{'value': 'INSPIRE-00133260', 'schema': 'INSPIRE ID'}],
  'name': 'Valle, Jose W.F.',
  'hidden': False,
  'record': {'$ref': 'https://inspirehep.net/api/authors/985058'},
  'degree_type': 'phd',
  'curated_relation': False},
 {'name': 'Ponce, William A.',
  'hidden': False,
  'degree_type': 'master',
  'curated_relation': False}]

In [177]:
r.get('positions')[6]

{'rank': 'PHD',
 'hidden': False,
 'record': {'$ref': 'https://inspirehep.net/api/institutions/907907'},
 'current': False,
 'end_date': '2001',
 'start_date': '1997',
 'institution': 'Valencia U., IFIC',
 'curated_relation': True}

In [179]:
pre_students

In [180]:
#def get_studens()
# TEMPORAL
try:
    pre_students
except NameError:
    pre_students=[]
#if not r.get('advisors'):
#    for p in r.get('positions'):

if any([ any([ss.get('student_profile_id')==profile_id for ss in s.get('students')]) 
   for s in pre_students]):
    print('Already in students')
    #return pre_students


In [181]:
for a in r.get('advisors'):
    # Get profile_id or author_id or name of advisor
    try:
        advisor_profile_id=a.get('record').get('$ref').split('/')[-1]
    except:
        advisor_profile_id=''

    stdlist=[]
    for old_a in pre_students:
        # TODO: check for other indetifiers
        if old_a.get('profile_id')==advisor_profile_id:
            stdlist=old_a.get('students')



    #raise Exception('No profile_id')
    # TODO Improve checkpoint
    print(a)
    if r.get('students'):
        d=r.get('students')
    else:
        d={}
        d['students']=[]
    d=get_advisor(d,advisor_profile_id,a,df)

    ds=get_students(r,a)

    if ds not in d['students']:
        d['students'].append(ds)
    pre_students.append(d)

pre_students

{'ids': [{'value': 'INSPIRE-00133260', 'schema': 'INSPIRE ID'}], 'name': 'Valle, Jose W.F.', 'hidden': False, 'record': {'$ref': 'https://inspirehep.net/api/authors/985058'}, 'degree_type': 'phd', 'curated_relation': False}
{'name': 'Ponce, William A.', 'hidden': False, 'degree_type': 'master', 'curated_relation': False}


[{'students': [{'degree_type': 'phd',
    'rank': 'PHD',
    'hidden': False,
    'institution_id': '907907',
    'current': False,
    'end_date': '2001',
    'start_date': '1997',
    'institution': 'Valencia U., IFIC',
    'curated_relation': True,
    'student_id': 'D.Restrepo.1',
    'student_name': 'Restrepo Quintero, Diego Alejandro',
    'student_profile_id': '991924'}],
  'profile_id': '985058',
  'name': 'Valle, Jose W.F.'},
 {'students': [{'degree_type': 'master',
    'rank': 'MASTER',
    'hidden': False,
    'institution_id': '903906',
    'current': False,
    'end_date': '1997',
    'start_date': '1995',
    'institution': 'Antioquia U.',
    'curated_relation': True,
    'student_id': 'D.Restrepo.1',
    'student_name': 'Restrepo Quintero, Diego Alejandro',
    'student_profile_id': '991924'}],
  'profile_id': '',
  'name': 'Ponce, William A.'}]

In [103]:
advisor=[]
for old_a in pre_students:
        # TODO: check for other indetifiers
        if old_a.get('profile_id')==advisor_profile_id:
            advisor=old_a.copy()
if advisor:


In [105]:
advisor

{'students': [{'degree_type': 'phd',
   'rank': 'PHD',
   'hidden': False,
   'institution_id': 907907,
   'current': False,
   'end_date': '2001',
   'start_date': '1997',
   'institution': 'Valencia U., IFIC',
   'curated_relation': True,
   'student_id': 'D.Restrepo.1',
   'student_name': 'Restrepo Quintero, Diego Alejandro',
   'student_profile_id': '991924'}],
 'profile_id': '985058',
 'name': 'Valle, Jose W.F.'}

In [99]:
stdlist

[{'degree_type': 'phd',
  'rank': 'PHD',
  'hidden': False,
  'institution_id': 907907,
  'current': False,
  'end_date': '2001',
  'start_date': '1997',
  'institution': 'Valencia U., IFIC',
  'curated_relation': True,
  'student_id': 'D.Restrepo.1',
  'student_name': 'Restrepo Quintero, Diego Alejandro',
  'student_profile_id': '991924'}]

True

In [21]:
del pre_students

In [187]:
d

{'author_id': 0,
 'name': 'Ponce, William A.',
 'student': {'degree_type': 'master',
  'rank': 'MASTER',
  'hidden': False,
  'institution_id': 903906,
  'current': False,
  'end_date': '1997',
  'start_date': '1995',
  'institution': 'Antioquia U.',
  'curated_relation': True}}

In [197]:
d

{'author_id': 993016,
 'name': 'Ponce, William A.',
 'student': {'degree_type': 'master',
  'rank': 'MASTER',
  'hidden': False,
  'institution_id': 903906,
  'current': False,
  'end_date': '1997',
  'start_date': '1995',
  'institution': 'Antioquia U.',
  'curated_relation': True}}

In [10]:
ca=set(df[df.institution_id==903002].author_id.unique())

Some time was a cinvestav author

In [25]:
cv=df[df.author_id.apply(lambda x: x in ca)].reset_index(drop=True)
i=cv.author_id.unique()[1]

In [27]:
i

988155

In [109]:
df['advisor']=df.author_id.apply(get_advisor)

KeyboardInterrupt: 

In [35]:
cv['get_advisor']=cv.author_id.apply(get_advisor)

In [59]:
cv_gr=cv[cv['get_advisor'].apply(lambda l: [int(d.get('record').get('$ref').split('/')[-1])
 for d in l if d.get('record')  ] if l else []).apply(lambda l: set(l).intersection(ca)).apply(len)>0]

In [74]:
cv_gr=cv_gr.drop_duplicates(subset=['author_id'])

In [77]:
cv_gr['full_name']=cv_gr.name.str['value']

In [84]:
cv_gr['advisor_id']=cv_gr['get_advisor'].apply(lambda l: [int(d.get('record').get('$ref').split('/')[-1])
 for d in l if d.get('record')  ] if l else []).apply(lambda l: set(l).intersection(ca)).apply(list)

In [95]:
cv_gr['advisor_name']=cv_gr.apply(lambda r: ([d.get('name') for d in r.get('get_advisor')
                          if int(d.get('record').get('$ref').split('/')[-1]) in r.get('advisor_id')
                         ]) , axis='columns' )

In [103]:
def get_name(i):
    return cv[cv['author_id']==i].iloc[0].get('name').get('value')

In [97]:
cv_gr=cv_gr.reset_index(drop=True)

In [105]:
cv_gr['advisor_name']=cv_gr['advisor_id'].apply(lambda l: [get_name(i) for i in l])

In [106]:
cv_gr[['full_name','advisor_name','advisor_id']]

,full_name,advisor_name,advisor_id
0,"Perez-Lorenzana, Abdel","[Zepeda, Arnulfo]",[982258]
1,"Montesinos-Velasquez, Merced","[Matos, Tonatiuh]",[998375]
2,"Parada Valencia, Alexander","[Miranda, Omar Gustavo]",[997309]
3,"Fernandez C, David J.","[Mielnik, Bogdan]",[1055726]
4,"González, Diego","[Montesinos-Velasquez, Merced]",[996953]
5,"Garcia-Salcedo, Ricardo","[Breton, Nora]",[1015335]
6,"Ayón-Beato, Eloy","[Garcia Diaz, Alberto A.]",[1008895]
7,"Breton, Nora","[Garcia Diaz, Alberto A.]",[1008895]
8,"Martinez Hernandez, Mario Ivan","[Herrera Corral, Gerardo Antonio]",[1006138]
9,"Rosas-Ortiz, Oscar","[Fernandez C, David J.]",[1010144]
